### Optimization of Sensory Neuron Properties
#### _In a 1-D Sensor-Mover_

In general, sensory neurons transduce an external stimuli into a neural response. Examples of sensory neurons in humans include cells of the retina, olfactory bulb, and the cochlea, to name a few. In this model the sensory neurons are modeled as simplified olfaction (smell) neurons. Mammalian olfactory research suggests the neuronal output frequency response has a rough linear relationship with the $\log_{10}$ of the external stimuli. The properties of the sensory neuron are as follows:
- Minimum output response
- Maximum output response
- Slope of encoding: how quickly the stimu
- Center point: point where field strength results in mid-output between minimum and maximum responses

To put this more formally, a stimuli-response functional relationship between the external stimuli strength, $|\phi|$, and the neuronal output, $y_{out}$ is defined.
$$y_{out} = f(|\phi|)$$ $$y \in [0, 1]$$

The function, $f$, is bound by a few principles. At sufficiently low external stimuli the output should be low, approaching a minimum, and the converse is true, that is the higher the stimuli the output should approach the maximum response. In this model the neuronal frequency response is analgous to the magnitude of the neuronal output.


In [14]:
%matplotlib inline
from pyBrainNetSim.models import world, individuals, network
from pyBrainNetSim.simulation import simnetwork
from pyBrainNetSim.generators import network as gennwk, builtin
import pyBrainNetSim.generators.settings.base as settings
import pyBrainNetSim.simulation.evolution as evo
import pyBrainNetSim.drawing.viewers as vw
import numpy as np
import pandas as pd
# from pyBrainNetSim.drawing.viewers import draw_networkx
# from pyBrainNetSim.generators import special_functions as sf
from bokeh.plotting import figure, show
from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import ColumnDataSource, Range1d, LinearAxis, HoverTool
from bokeh.layouts import Row
output_notebook()


Loading BokehJS ...

### "Efficient" Example Network
Example of approximate sensory neuron properties. The values were determined via trial and error and may not be the optimal ones. 

In [37]:
min_out, max_out, mid_stim, slope = 0, 1, 0.3, 2.0  # De

settings.NODE_PROPS['Internal']['spontaneity'] = 0.0
settings.NODE_PROPS['Internal']['energy_basal_rate'] = settings.NODE_PROPS['Motor']['energy_basal_rate'] = settings.NODE_PROPS['Sensory']['energy_basal_rate'] = 0.1
settings.NODE_PROPS['Internal']['energy_value'] = 30.
settings.NODE_PROPS['Motor']['energy_value'] = 30.
settings.NODE_PROPS['Sensory']['energy_value'] = 30.

neurons, edges = builtin.simple_SM1D(number_signal_discretization=20, sens_sensitivity=slope, sens_mid=mid_stim, sens_minsignal=min_out, sens_maxsignal=max_out )
smpd = gennwk.SensorMoverProperties(nodes=neurons, edges=edges, internal=gennwk.InternalNodeProperties(settings))
G = network.NeuralNetData(smpd.create_digraph())

w = world.Environment(origin=[-10], max_point=[10], field_permeability=0.1, field_decay=world.ExponentialDecayScalarField)
food = world.Attractor(environment=w, position=[10], strength=1., energy_value=10*len(neurons))  # add "food"
I = individuals.SensorMover(w, initial_network=simnetwork.HebbianNetworkBasic(G, pos_synapse_growth=0., neg_synapse_growth=0.), position=[-5.], data_cutoff=None)
I.sim_time_steps(max_iter=60)

6368 FOUND s0 TARGET @ [10] @ t:20
6368 FOUND s1 TARGET @ [-5] @ t:39
6368 FOUND s3 TARGET @ [5] @ t:57
6368 FOUND s5 TARGET @ [6] @ t:58


In [38]:
n=42
print I.environment.attractors[I.environment.attractors.keys()[0]].position
print I.internal.simdata[n].node['M0']['force_direction'], I.internal.simdata[n].node['S0']['sensor_direction']
fig = vw.draw_fields(w, sensory_neuron=G.node['S0'])
_, f_int1 = vw.draw_network_bokeh(I.internal.simdata[n])
_, f_int2 = vw.draw_network_bokeh(I.internal.simdata[n+1])
_, f_int3 = vw.draw_network_bokeh(I.internal.simdata[n+2])
t1 = vw.draw_trajectory(I)
show(Row(fig, t1))
show(Row(f_int1, f_int2, f_int3))
# show(t1)

[7]
[-1] [-1.0]


### Optimization using "Survival of the Fittest" and No Sexual Reproduction
Solve for optimum sensory neuron properties using population without sexual reproduction. Those configurations that fail to consistently find food are naturally eliminated.

In [4]:
import scipy.stats as stats
min_smid, max_smid, min_ssen, max_ssen = 0.05, 0.5, 0.5, 2.
neurons, edges = builtin.simple_SM1D(number_signal_discretization=8)
settings.NODE_PROPS['Sensory']['number_of_nodes'] = settings.NODE_PROPS['Motor']['number_of_nodes'] = 2
settings.NODE_PROPS['Sensory']['sensory_mid'] = stats.uniform(loc=min_smid, scale=max_smid - min_smid)
settings.NODE_PROPS['Sensory']['stimuli_sensitivity'] = stats.uniform(loc=min_ssen, scale=max_ssen - min_ssen)
reproductive_cost = 1.0  # i.e. reproduction would use 100% of its energy, so reproduction is inhibited.

smpd = gennwk.SensorMoverProperties(nodes=neurons, edges=edges, sensory_dist=gennwk.SensoryNodeProperties(settings))
smp = evo.SensorMoverPopulation(w, smpd, initial_population_size=1, reproductive_cost=reproductive_cost, data_cutoff=None, pos_synapse_growth=0., neg_synapse_growth=0.)
smp.sim_time_steps(max_iter=70)

### Visualize the Sampled Sensory Properties

In [5]:
size_base = 10.
def get_ploting_data(sm_pop, ix, e_initial):
    smid = [i.internal.simdata[ix].node['S0']['sensory_mid'] for iid, i in sm_pop.individuals.iteritems()]
    ssen = [i.internal.simdata[ix].node['S0']['stimuli_sensitivity'] for iid, i in sm_pop.individuals.iteritems()]
    ef = [i.internal.simdata[ix].total_energy/e_initial for iid, i in sm_pop.individuals.iteritems()]
    csize = [size_base * i.internal.simdata[ix].total_energy/e_initial for iid, i in sm_pop.individuals.iteritems()]
    iid = [iid for iid in sm_pop.individuals.iterkeys()]
    return ColumnDataSource(data=dict(iid=iid, sm=smid, ss=ssen, ef=ef, cs=csize))

def plot_props(sm_pop):
    e_initial = (sm_pop.energy_initial.iloc[0] - sm_pop.energy_added.iloc[0])/len(smp.individuals)
    data_i = get_ploting_data(sm_pop, 0, e_initial)
    data_f = get_ploting_data(sm_pop, -1, e_initial)
    hover_i = HoverTool(tooltips=[("Individual", "@iid"), ("Mid-Sensor Transduction", "@sm"), ("Sensitivity", "@ss"), ("Energy Fraction", "@ef")])
    p_i = figure(title="Sensory Neuron Properties @t 0 (%s individuals)" %sm_pop.balance['# Init Agents'].iloc[0], tools=["crosshair,pan,reset,save,wheel_zoom", hover_i], x_axis_label='Sensory Mid-Point Transduction', y_axis_label='Sensitivity',
               x_range=Range1d(min_smid, max_smid), y_range=Range1d(min_ssen, max_ssen))
    p_i.circle("sm", "ss", source=data_i, size="cs", color='navy', alpha=0.5)

    hover_f = HoverTool(tooltips=[("Individual", "@iid"), ("Mid-Sensor Transduction", "@sm"), ("Sensitivity", "@ss"), ("Energy Fraction", "@ef")])
    p_f = figure(title="Sensory Neuron Properties @t %s (%s individuals)" %(smp.t, len(sm_pop.individuals)), tools=["crosshair,pan,reset,save,wheel_zoom", hover_f], x_axis_label='Sensory Mid-Point Transduction', y_axis_label='Sensitivity',
               x_range=Range1d(min_smid, max_smid), y_range=Range1d(min_ssen, max_ssen))
    p_f.circle("sm", "ss", source=data_f, size="cs", color='navy', alpha=0.5)
    return p_i, p_f

pi, pf = plot_props(smp)
show(Row(pi, pf))
smp.individual_energy_initial.tail()
smp.balance.iloc[0:10]

,# Init Agents,# Born,# Died,# Final Agents,# Agents,Initial E,Final E,Consumed E,Lost E,Added E,Global E
0,1,0,0,1,1,1620.0,1612.6,7.4,-0.0,0.0,1620.0
1,1,0,0,1,1,1612.6,1600.2,12.4,-0.0,0.0,1620.0
2,1,0,0,1,1,1600.2,1577.8,22.4,-0.0,0.0,1620.0
3,1,0,0,1,1,1577.8,1554.4,23.4,-0.0,0.0,1620.0
4,1,0,0,1,1,1554.4,1530.0,24.4,-0.0,0.0,1620.0
5,1,0,0,1,1,1530.0,1505.6,24.4,-0.0,0.0,1620.0
6,1,0,0,1,1,1505.6,1481.2,24.4,-0.0,0.0,1620.0
7,1,0,0,1,1,1481.2,1457.8,23.4,-0.0,0.0,1620.0
8,1,0,0,1,1,1457.8,1436.4,21.4,-0.0,0.0,1620.0
9,1,0,0,1,1,1436.4,1416.0,20.4,-0.0,0.0,1620.0


In [6]:
I = smp.individuals['G0_I0']
fig = vw.draw_fields(w, sensory_neuron=I.internal.simdata[0].node['S0'])
_, f_int = vw.draw_network_bokeh(I.internal.simdata[20])
t1 = vw.draw_trajectory(I)
e = I.energy_initial.sum(axis=1)
# print e.index.values
e1 = figure()
e1.line(x=e.index.values, y=e.values)
# I.energy_initial.sum(axis=1)

show(Row(fig, f_int, t1))
show(Row(e1))

### Optimization using "Survival of the Fittest" and Sexual Reproduction
Solve for optimum sensory neuron properties using population with sexual reproduction. Those configurations that fail to consistently find food are naturally eliminated. Potentially new configurations combine sensory neuron properties.

In [7]:
smp2 = evo.SensorMoverPopulation(w, smpd, initial_population_size=20, reproductive_cost=0.01, reproductive_threshold=10, reproductive_mode='sexual', data_cutoff=3, pos_synapse_growth=0., neg_synapse_growth=0.)
smp2.sim_time_steps(max_iter=100)

G0_I16 FOUND s7 TARGET @ [-10] @ t:0


KeyboardInterrupt: 

In [ ]:
pi, pf = plot_props(smp2)
show(Row(pi, pf))